In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [18]:
#RUN ONLY ONCE
# %pip install kaggle
# !mkdir -p ~/.kaggle
#before running the code below, make sure to download Kaggle API Token first and change path to where the file was downloaded
# !mv /path/to/downloaded/kaggle.json ~/.kaggle/

# import kaggle
# !kaggle datasets download -d nelgiriyewithana/credit-card-fraud-detection-dataset-2023
# !kaggle datasets download -d praveengovi/credit-risk-classification-dataset

# import zipfile
# with zipfile.ZipFile('credit-card-fraud-detection-dataset-2023.zip', 'r') as zip_ref:
#     zip_ref.extractall('credit-card-fraud-detection-dataset')
# with zipfile.ZipFile('credit-risk-classification-dataset.zip', 'r') as zip_ref:
#     zip_ref.extractall('credit-risk-classification-dataset')
# !del credit-card-fraud-detection-dataset-2023.zip
# !del credit-risk-classification-dataset.zip

In [85]:
#data loader methods
def load_credit_risk():
    payment = pd.read_csv('credit-risk-classification-dataset/payment_data.csv')
    customer = pd.read_csv('credit-risk-classification-dataset/customer_data.csv', usecols=range(0,2))
    merged_data = payment.merge(customer, left_on='id', right_on='id').values
    
    cols_to_drop = [0,8,11]#dropping the id an dates; not helpful
    data = np.delete(merged_data, cols_to_drop, axis=1)
    return (data[:,:9], data[:,-1])
    

def load_credit_fraud():
    #deleting the 1st col as it is just the id; X is 1-30 and y is col 31
    X = pd.read_csv('credit-card-fraud-detection-dataset/creditcard_2023.csv', usecols=range(1,31)).values
    return (X[:,:-1], X[:,-1].astype(int))